In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys, os, gc

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *

In [3]:
# import aggregated trips
trips_aggregated_df_init = pd.read_parquet(TRIPS_GROUPED_SPATIO_TEMPORAL_PATH)
trips_aggregated_df = trips_aggregated_df_init[
    (trips_aggregated_df_init['h3_res'] != TUNE_H3_RESOLUTION) & 
    (trips_aggregated_df_init['time_interval_length'] != TUNE_TIME_INTERVAL_LENGTH)
]
trips_aggregated_df_reduced = trips_aggregated_df_init[
    (trips_aggregated_df_init['h3_res'] == TUNE_H3_RESOLUTION) & 
    (trips_aggregated_df_init['time_interval_length'] == TUNE_TIME_INTERVAL_LENGTH)
]
trips_aggregated_df_reduced = trips_aggregated_df_reduced.reset_index(drop=True)

In [4]:
print(f"Total number of entries in the demand dataset for all h3 resolutions and all time intervals: {trips_aggregated_df_init.index.size}")
print(f"Total number of entries in the demand dataset for all h3 resolutions and all time intervals: {trips_aggregated_df.index.size}")
print(f"Total number of entries in the demand dataset for all h3 resolutions and all time intervals: {trips_aggregated_df_reduced.index.size}")

Total number of entries in the demand dataset for all h3 resolutions and all time intervals: 4253147
Total number of entries in the demand dataset for all h3 resolutions and all time intervals: 2055463
Total number of entries in the demand dataset for all h3 resolutions and all time intervals: 374168


In [5]:
# add time features
trips_aggregated_df['hour'] = trips_aggregated_df.datetime_start_floored.dt.hour
trips_aggregated_df['weekday'] = trips_aggregated_df.datetime_start_floored.dt.weekday
trips_aggregated_df['month'] = trips_aggregated_df.datetime_start_floored.dt.month

trips_aggregated_df_reduced['hour'] = trips_aggregated_df_reduced.datetime_start_floored.dt.hour
trips_aggregated_df_reduced['weekday'] = trips_aggregated_df_reduced.datetime_start_floored.dt.weekday
trips_aggregated_df_reduced['month'] = trips_aggregated_df_reduced.datetime_start_floored.dt.month

trips_aggregated_df_reduced.head(2)

C:\Users\Kateryna Kuian\AppData\Local\Temp\ipykernel_17416\3897331288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_aggregated_df['hour'] = trips_aggregated_df.datetime_start_floored.dt.hour
C:\Users\Kateryna Kuian\AppData\Local\Temp\ipykernel_17416\3897331288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_aggregated_df['weekday'] = trips_aggregated_df.datetime_start_floored.dt.weekday
C:\Users\Kateryna Kuian\AppData\Local\Temp\ipykernel_17416\3897331288.py:4: SettingWithCopyWarning: 

,datetime_start_floored,start_hex_id,end_hex_id,demand,h3_res,time_interval_length,hour,weekday,month
0,2019-01-20,881f1a1601fffff,881f1a1659fffff,3,8,6,0,6,1
1,2019-01-20,881f1a1609fffff,881f1a1643fffff,2,8,6,0,6,1


In [6]:
# #
# trips_aggregated_df['hour_sin'] = np.sin(trips_aggregated_df.hour*(2.*np.pi/24))
# trips_aggregated_df['hour_cos'] = np.cos(trips_aggregated_df.hour*(2.*np.pi/24))
# # m
# trips_aggregated_df['month_sin'] = np.sin((trips_aggregated_df.month-1)*(2.*np.pi/12))
# trips_aggregated_df['month_cos'] = np.cos((trips_aggregated_df.month-1)*(2.*np.pi/12))

In [7]:
# convert time features to dummy variables (one-hot encoding) and replace the original features
hour_dummies = pd.get_dummies(trips_aggregated_df['hour']).add_prefix('hour_')
weekday_dummies = pd.get_dummies(trips_aggregated_df['weekday']).add_prefix('weekday_')
month_dummies = pd.get_dummies(trips_aggregated_df['month']).add_prefix('month_')

trips_aggregated_df.drop(['hour', 'weekday', 'month'], axis=1, inplace=True)
trips_aggregated_df = pd.concat([trips_aggregated_df, hour_dummies, weekday_dummies, month_dummies], axis=1)

hour_dummies_reduced = pd.get_dummies(trips_aggregated_df_reduced['hour']).add_prefix('hour_')
weekday_dummies_reduced = pd.get_dummies(trips_aggregated_df_reduced['weekday']).add_prefix('weekday_')
month_dummies_reduced = pd.get_dummies(trips_aggregated_df_reduced['month']).add_prefix('month_')

trips_aggregated_df_reduced.drop(['hour', 'weekday', 'month'], axis=1, inplace=True)
trips_aggregated_df_reduced = pd.concat([trips_aggregated_df_reduced, hour_dummies_reduced, weekday_dummies_reduced, month_dummies_reduced], axis=1)
trips_aggregated_df_reduced.head(2)

C:\Users\Kateryna Kuian\AppData\Local\Temp\ipykernel_17416\3029032996.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_aggregated_df.drop(['hour', 'weekday', 'month'], axis=1, inplace=True)


,datetime_start_floored,start_hex_id,end_hex_id,demand,h3_res,time_interval_length,hour_0,hour_6,hour_12,hour_18,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2019-01-20,881f1a1601fffff,881f1a1659fffff,3,8,6,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-01-20,881f1a1609fffff,881f1a1643fffff,2,8,6,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
del hour_dummies, weekday_dummies, month_dummies, hour_dummies_reduced, weekday_dummies_reduced, month_dummies_reduced
gc.collect()

593

In [9]:
# load the weather dataframe, reduce data types
weather_df = pd.read_parquet(WEATHER_AGGR_TEMPORAL_PATH)
weather_df = weather_df.astype(np.float16)
weather_df['time_interval_length'] = weather_df.time_interval_length.astype(np.uint8)
weather_df.head(2)

,min_temperature,max_temperature,mean_temperature,mean_mean_wind_speed,mean_total_cloud_cover,sum_precipitation,time_interval_length
MESS_DATUM,,,,,,,
2019-01-01 00:00:00,7.398438,7.398438,7.398438,2.800781,8.0,0.0,1
2019-01-01 01:00:00,7.699219,7.699219,7.699219,2.900391,8.0,0.0,1


In [10]:
# merge weather data with trips data
trips_aggregated_df = pd.merge(trips_aggregated_df, weather_df,  how='left', 
                    left_on=['datetime_start_floored','time_interval_length'],
                    right_on = ['MESS_DATUM','time_interval_length'])

trips_aggregated_df_reduced = pd.merge(trips_aggregated_df_reduced, weather_df,  how='left', 
                    left_on=['datetime_start_floored','time_interval_length'],
                    right_on = ['MESS_DATUM','time_interval_length'])

In [11]:
del weather_df
gc.collect()

0

In [12]:
# remove datetime_start_floored column as it won't be needed anymore
trips_aggregated_df = trips_aggregated_df.drop(columns=['datetime_start_floored'])
trips_aggregated_df_reduced = trips_aggregated_df_reduced.drop(columns=['datetime_start_floored'])

In [13]:
# load poi data and reduce datatypes
hexagons_with_pois_df = pd.read_parquet(HEXAGON_WITH_POIS_PATH)
hexagons_with_pois_df = hexagons_with_pois_df.drop(columns=["hex_and_neighbors"])
hexagons_with_pois_df = hexagons_with_pois_df.astype({
    "h3_res": np.uint16,
    "sustenance_poi": np.uint16,
    "public_transport_poi": np.uint16,
    "education_poi": np.uint16,
    "arts_and_culture_poi": np.uint16,
    "sports_poi": np.uint16,
})

In [14]:
# add poi use data as feature of the start location
trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_pois_df.drop(columns=["h3_res"]), left_on="start_hex_id", right_on="hex")
trips_aggregated_df = trips_aggregated_df.drop(columns={"hex"})

# add '_start' suffix to poi columns
trips_aggregated_df = trips_aggregated_df.rename(columns={
    "sustenance_poi": "sustenance_poi_start",
    "public_transport_poi": "public_transport_poi_start",
    "education_poi": "education_poi_start",
    "arts_and_culture_poi": "arts_and_culture_poi_start",
    "sports_poi": "sports_poi_start",
})

# add poi use data as feature of the start location
trips_aggregated_df_reduced = pd.merge(trips_aggregated_df_reduced, hexagons_with_pois_df.drop(columns=["h3_res"]), left_on="start_hex_id", right_on="hex")
trips_aggregated_df_reduced = trips_aggregated_df_reduced.drop(columns={"hex"})

# add '_start' suffix to poi columns
trips_aggregated_df_reduced = trips_aggregated_df_reduced.rename(columns={
    "sustenance_poi": "sustenance_poi_start",
    "public_transport_poi": "public_transport_poi_start",
    "education_poi": "education_poi_start",
    "arts_and_culture_poi": "arts_and_culture_poi_start",
    "sports_poi": "sports_poi_start",
})
trips_aggregated_df_reduced.head(2)

,start_hex_id,end_hex_id,demand,h3_res,time_interval_length,hour_0,hour_6,hour_12,hour_18,weekday_0,...,max_temperature,mean_temperature,mean_mean_wind_speed,mean_total_cloud_cover,sum_precipitation,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start
0,881f1a1601fffff,881f1a1659fffff,3,8,6,1,0,0,0,0,...,-6.199219,-7.699219,0.766602,0.000000,0.0,82,80,11,5,5
1,881f1a1601fffff,881f1ab923fffff,1,8,6,0,0,0,1,0,...,-5.000000,-6.699219,0.433350,0.333252,0.0,82,80,11,5,5


In [15]:
trips_aggregated_df_reduced.columns

Index(['start_hex_id', 'end_hex_id', 'demand', 'h3_res',
       'time_interval_length', 'hour_0', 'hour_6', 'hour_12', 'hour_18',
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12', 'min_temperature', 'max_temperature',
       'mean_temperature', 'mean_mean_wind_speed', 'mean_total_cloud_cover',
       'sum_precipitation', 'sustenance_poi_start',
       'public_transport_poi_start', 'education_poi_start',
       'arts_and_culture_poi_start', 'sports_poi_start'],
      dtype='object')

In [16]:
# add poi use data as feature of the end location
trips_aggregated_df_reduced = pd.merge(trips_aggregated_df_reduced, hexagons_with_pois_df, left_on="end_hex_id", right_on="hex")
trips_aggregated_df_reduced = trips_aggregated_df_reduced.drop(columns={"hex", "h3_res_y"})

# add '_end' suffix to poi columns
trips_aggregated_df_reduced = trips_aggregated_df_reduced.rename(
    columns={
        "sustenance_poi": "sustenance_poi_end",
        "public_transport_poi": "public_transport_poi_end",
        "education_poi": "education_poi_end",
        "arts_and_culture_poi": "arts_and_culture_poi_end",
        "sports_poi": "sports_poi_end",
    }
)
trips_aggregated_df_reduced.head(2)

,start_hex_id,end_hex_id,demand,h3_res_x,time_interval_length,hour_0,hour_6,hour_12,hour_18,weekday_0,...,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start,sustenance_poi_end,public_transport_poi_end,education_poi_end,arts_and_culture_poi_end,sports_poi_end
0,881f1a1601fffff,881f1a1659fffff,3,8,6,1,0,0,0,0,...,82,80,11,5,5,162,93,23,9,16
1,881f1a1601fffff,881f1a1659fffff,1,8,6,0,0,1,0,0,...,82,80,11,5,5,162,93,23,9,16


In [17]:
del hexagons_with_pois_df
gc.collect()

0

In [18]:
# load land use data and reduce data types
hexagons_with_land_use_df = pd.read_parquet(HEXAGONS_WITH_LAND_USE_PATH)
hexagons_with_land_use_df = hexagons_with_land_use_df.astype(np.float16)
hexagons_with_land_use_df.head(2)

land_use,land_use_1,land_use_2,land_use_3,land_use_4,land_use_5,land_use_6,land_use_7,land_use_8,land_use_9,land_use_10,...,land_use_13,land_use_14,land_use_15,land_use_16,land_use_17,land_use_18,land_use_19,land_use_20,land_use_21,land_use_22
hexagon_id,,,,,,,,,,,,,,,,,,,,,
871f1a140ffffff,0.000000,0.000000,0.000000,0.000000,0.006058,0.000000,0.019272,0.015480,0.000000,0.319092,...,0.000000,0.0,0.0,0.007874,0.0,0.000000,0.0,0.001923,0.008865,0.0
871f1a144ffffff,0.148926,0.155518,0.016571,0.084351,0.135986,0.058533,0.071045,0.082703,0.004414,0.031219,...,0.000406,0.0,0.0,0.000000,0.0,0.004196,0.0,0.070251,0.000000,0.0


In [19]:
# add land use data as feature of the start location
trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_land_use_df.add_prefix("start_"), left_on="start_hex_id", right_on="hexagon_id")
trips_aggregated_df_reduced = pd.merge(trips_aggregated_df_reduced, hexagons_with_land_use_df.add_prefix("start_"), left_on="start_hex_id", right_on="hexagon_id")
trips_aggregated_df_reduced.head(2)

,start_hex_id,end_hex_id,demand,h3_res_x,time_interval_length,hour_0,hour_6,hour_12,hour_18,weekday_0,...,start_land_use_13,start_land_use_14,start_land_use_15,start_land_use_16,start_land_use_17,start_land_use_18,start_land_use_19,start_land_use_20,start_land_use_21,start_land_use_22
0,881f1a1601fffff,881f1a1659fffff,3,8,6,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066467,0.0,0.0
1,881f1a1601fffff,881f1a1659fffff,1,8,6,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066467,0.0,0.0


In [20]:
# add land use data as feature of the end location
trips_aggregated_df_reduced = pd.merge(trips_aggregated_df_reduced, hexagons_with_land_use_df.add_prefix("end_"), left_on="end_hex_id", right_on="hexagon_id")
trips_aggregated_df_reduced.head(2)

,start_hex_id,end_hex_id,demand,h3_res_x,time_interval_length,hour_0,hour_6,hour_12,hour_18,weekday_0,...,end_land_use_13,end_land_use_14,end_land_use_15,end_land_use_16,end_land_use_17,end_land_use_18,end_land_use_19,end_land_use_20,end_land_use_21,end_land_use_22
0,881f1a1601fffff,881f1a1659fffff,3,8,6,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.147339,0.0,0.0
1,881f1a1601fffff,881f1a1659fffff,1,8,6,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.147339,0.0,0.0


In [21]:
del hexagons_with_land_use_df
gc.collect()

0

In [22]:
print(trips_aggregated_df.memory_usage(index=True, deep=True).sum() / 1024**2)
print(trips_aggregated_df_reduced.memory_usage(index=True, deep=True).sum() / 1024**2)

529.0944137573242
110.70897674560547


In [23]:
start_hex_id_map = dict(enumerate(trips_aggregated_df.start_hex_id.unique()))
start_hex_id_map = {v: k for k, v in start_hex_id_map.items()}

reduced_start_hex_id_map = dict(enumerate(trips_aggregated_df_reduced.start_hex_id.unique()))
reduced_end_hex_id_map = dict(enumerate(trips_aggregated_df_reduced.end_hex_id.unique()))
reduced_start_hex_id_map = {v: k for k, v in reduced_start_hex_id_map.items()}
reduced_end_hex_id_map = {v: k for k, v in reduced_end_hex_id_map.items()}

In [24]:
trips_aggregated_df["start_hex_id"] = trips_aggregated_df["start_hex_id"].map(start_hex_id_map).astype(np.uint16)

trips_aggregated_df_reduced["start_hex_id"] = trips_aggregated_df_reduced["start_hex_id"].map(reduced_start_hex_id_map).astype(np.uint16)
trips_aggregated_df_reduced["end_hex_id"] = trips_aggregated_df_reduced["end_hex_id"].map(reduced_end_hex_id_map).astype(np.uint16)

In [25]:
# store all model data in one file
trips_aggregated_df.to_feather(MODEL_DATA_PATH)
trips_aggregated_df_reduced.to_feather(REDUCED_MODEL_DATA_PATH)

In [26]:
if not os.path.exists(MODEL_DATA_DIR_PATH):
	os.makedirs(MODEL_DATA_DIR_PATH)

# store model data for each time and hex resolution in one file
# additionally create dummies for start and end hexagons  
for h3_res in CALC_H3_RESOLUTIONS:
    for t in CALC_TIME_INTERVAL_LENGTHS:
        if h3_res == TUNE_H3_RESOLUTION and t == TUNE_TIME_INTERVAL_LENGTH: continue

        model_data = trips_aggregated_df[
            (trips_aggregated_df.h3_res == h3_res) & (trips_aggregated_df.time_interval_length == t)
        ].reset_index(drop=True)

        if t > 5:
            model_data = model_data.drop(columns=["mean_temperature"])
        else:
            model_data = model_data.drop(columns=["min_temperature", "max_temperature"])

        start_hex_dummies = pd.get_dummies(model_data.start_hex_id, prefix="start_")
        model_data = pd.concat([model_data, start_hex_dummies], axis=1)
        model_data = model_data.drop(columns=['start_hex_id', 'end_hex_id'])

        model_data.to_feather(os.path.join(MODEL_DATA_DIR_PATH, f"{h3_res}_{t}.feather"))

In [27]:
if t > TUNE_TIME_INTERVAL_LENGTH:
    trips_aggregated_df_reduced = trips_aggregated_df_reduced.drop(columns=["mean_temperature"])
else:
    trips_aggregated_df_reduced = trips_aggregated_df_reduced.drop(columns=["min_temperature", "max_temperature"])

start_hex_dummies = pd.get_dummies(trips_aggregated_df_reduced.start_hex_id, prefix="start_")
end_hex_dummies = pd.get_dummies(trips_aggregated_df_reduced.end_hex_id, prefix="end_")
trips_aggregated_df_reduced = pd.concat([trips_aggregated_df_reduced, start_hex_dummies, end_hex_dummies], axis=1)
trips_aggregated_df_reduced = trips_aggregated_df_reduced.drop(columns=['start_hex_id', 'end_hex_id'])

trips_aggregated_df_reduced.to_feather(os.path.join(MODEL_DATA_DIR_PATH, f"{TUNE_H3_RESOLUTION}_{TUNE_TIME_INTERVAL_LENGTH}.feather"))